<a href="https://colab.research.google.com/github/Igor-Daudt/Manim_jupiter_networks/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from manim import *
from pandas import read_csv
from math import floor
from random import sample, random

In [ ]:

class DataMatrix(VGroup):
    def __init__(
        self,
        data,
        num_rows,
        cell_width = 1.0,
        cell_height = 1.0,
        rows_margin = 0.0,
        cols_margin = 0.0,
        stroke_width = 2.0,
        font_size = 24,
        font_type = "Century",
        color_mode = False,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.num_rows, self.num_cols = num_rows, int(len(data) / num_rows)
        self.cell_width = cell_width
        self.cell_height = cell_height
        self.rows_margin = rows_margin
        self.cols_margin = cols_margin
        self.font_size = font_size
        self.font_type = font_type
        self.matrix_data = data.copy()
        self.stroke_width = stroke_width
        self.color_mode = color_mode

        self.build_matrix()

    def build_matrix(self):
        self.submobjects.clear()

        for i in range(self.num_rows):
            row = VGroup()
            for j in range(self.num_cols):
                row.add(Rectangle(
                    width = self.cell_width,
                    height = self.cell_height,
                    stroke_color = WHITE
                ).set_stroke(width = self.stroke_width))

                if self.color_mode:
                    row[j].set_fill(color = WHITE, opacity = self.matrix_data[i * self.num_cols + j])
                else:
                    row[j].add(Text(text = self.matrix_data[i * self.num_cols + j], font = self.font_type, font_size = self.font_size))

            row.arrange(RIGHT, buff = self.cols_margin)
            self.add(row)

        self.arrange(DOWN, buff = self.rows_margin)
        self.move_to(ORIGIN)

    def set_text_mode(self):
        self.color_mode = False
        self.build_matrix()

    def set_color_mode(self):
        self.color_mode = True
        self.build_matrix()

    def update_data(self, new_data):
        self.matrix_data = new_data
        self.build_matrix()

In [ ]:
def create_colored_neuron(radius_lenght = 1.0, color = BLUE, fill_color = WHITE, opacity = 0):
    return Circle(radius = radius_lenght, color = color).set_fill(color = fill_color, opacity = opacity)

def create_rectangle_with_text(text, font_type, font_size, side_length = 2.0, func_square = lambda x: x, func_text = lambda x: x):
    sq = Square(side_length = side_length)
    func_square(sq)

    text = Text(text = text, font=font_type, font_size = font_size)
    func_text(text)

    sq.add(text)
    return sq

def normalize_values(listRGB):
    return list(map(lambda x: round(x / 255, 1), listRGB))

def format_values(list_numbers, simplify = True):
    return list(map(lambda x: format_value(x, simplify), list_numbers))

def format_value(number, simplify):
    out = ""
    if number == 0:
        return "0"
    elif number == 1:
        return "1"
    elif simplify:
        out = str(number).replace("0.", "")
    else:
        out = str(number).replace(".", "")
    return f",{out}"

def sucession_animations(animation, object_list, delay):
    current_delay = 0
    sucessions_functions = list()
    for obj in object_list:
        sucessions_functions.append(Succession(
            Wait(current_delay),
            animation(obj)))
        current_delay += delay
    return sucessions_functions

def add_intervals(animations, delay = 0.5):
    output = []
    for anim in animations:
        output.append(Succession(
          anim,
          Wait(delay)))
    return output

def create_arranged_group(labels, mobject, direction = DOWN, **kwargs):
    return VGroup(mobject(i, **kwargs) for i in labels).arrange(direction)

# create_groups_connection: Each object of group 2 is connected to every object from group1
def create_groups_connection(group1, group2, func_reference1, func_reference2, **kwargs):
    output = VGroup()
    for i in group2:
        for j in group1:
            output.add(Line(func_reference1(j), func_reference2(i), **kwargs))

def animate_half_lines(lines, line_width = 2, run_time= 1.5, animation_time = 1):
    selected = sample(list(lines), k=len(lines)//2)

    animations = []
    for line in selected:
        highlight = line.copy()
        highlight.set_stroke(color=BLUE, width=line_width)

        anim = ShowPassingFlash(
            highlight,
            time_width=animation_time,
            run_time=run_time,
            rate_func = linear
        )
        animations.append(anim)

    return animations

def animate_neurons(circles, run_time = 1.5):
    animations = []
    for i in range(len(circles)):
        activation = pow(random(), 2)
        activated_circle = circles[i].copy()
        activated_circle.set_fill(color = WHITE, opacity = activation)

        animations.append(Transform(circles[i], activated_circle, run_time = run_time))

    return animations

def animate_output(circles, label, run_time = 1.5):
    animations = []
    for i in range(len(circles)):
        if i == label:
            activation = 1 - random() / 5
        else:
            activation = random() / 6
        activated_circle = circles[label].copy()
        activated_circle.set_fill(color = WHITE, opacity = activation)

        animations.append(Transform(circles[i], activated_circle, run_time = run_time))

    return animations




In [ ]:
# Global variables to control the view
FONT_SIZE_TITLE = 46
FONT_TYPE = "Go" # Century
NUM_ROWS_MNIST = 28 # 28
NUM_COLUMNS_MNIST = 28 # 28


# Variables to control the logic
csv_file = read_csv("sample_data/mnist_test.csv", header = None)
labels = csv_file.iloc[:10, 0].values.tolist()
images = csv_file.iloc[:10, 1:].values.tolist()

pixels_normalized = [normalize_values(i) for i in images]
format_pixels_values = [format_values(i) for i in pixels_normalized]

In [ ]:
%%manim NeuralNetwork

# Layers Constants
NUM_INPUTS_REPRESENTATIVE = 14
PERCEPTRONS_HIDDEN_LAYER1 = 8
PERCEPTRONS_HIDDEN_LAYER2 = 8
NUM_OUTPUTS = 10
INPUT_SIDE_SIZE = 0.3
INPUT_FONT_SIZE = 18
RADIUS_NEURONS = 0.25
STROKE_WEIGHT = 1
PADDING_SURROUNDING_BOXES = 0.2
LABELS_FONT_LAYERS = 20
COLOR_NEURONS = BLUE
COLOR_WEIGHT = WHITE
MARGIN_LEFT = 2
MARGIN_BETWEEN_LAYERS = 1.2
MARGIN_EXAMPLE_INPUT = 0.2
DISTANCE_BETWEEN_NEURONS = 0.15

SIZE_MNIST_EXAMPLE_SIDE = 0.05

class NeuralNetwork(Scene):
    def construct(self):
        # Layers Variables
        input_layer = VGroup()
        weights_hidden1= VGroup()
        hidden_layer1 = VGroup(create_colored_neuron(radius_lenght = RADIUS_NEURONS, color = COLOR_NEURONS) for i in range(PERCEPTRONS_HIDDEN_LAYER1)).arrange(DOWN, buff = DISTANCE_BETWEEN_NEURONS)
        weights_hidden2 = VGroup()
        hidden_layer2 = VGroup(create_colored_neuron(radius_lenght = RADIUS_NEURONS, color = COLOR_NEURONS) for i in range(PERCEPTRONS_HIDDEN_LAYER2)).arrange(DOWN, buff = DISTANCE_BETWEEN_NEURONS)
        weights_output = VGroup()
        output_layer = VGroup(create_colored_neuron(radius_lenght = RADIUS_NEURONS, color = COLOR_NEURONS) for i in range(NUM_OUTPUTS)).arrange(DOWN, buff = DISTANCE_BETWEEN_NEURONS)

        for i in range(NUM_INPUTS_REPRESENTATIVE):
            input_layer.add(create_rectangle_with_text(format_pixels_values[0][i], font_type = FONT_TYPE, font_size = INPUT_FONT_SIZE, side_length = INPUT_SIDE_SIZE))

            if floor(NUM_INPUTS_REPRESENTATIVE / 2) == i:
                input_layer[i] = MathTex(r"\vdots").scale(1.2)

        # Positioning layers
        input_layer.arrange(DOWN, buff = DISTANCE_BETWEEN_NEURONS).move_to(ORIGIN).to_edge(LEFT, buff = MARGIN_LEFT)
        hidden_layer1.next_to(input_layer,   RIGHT,  buff = MARGIN_BETWEEN_LAYERS)
        hidden_layer2.next_to(hidden_layer1, RIGHT,  buff = MARGIN_BETWEEN_LAYERS)
        output_layer.next_to(hidden_layer2,  RIGHT,  buff = MARGIN_BETWEEN_LAYERS)

        weights_hidden1.add(Line(j.get_right(), hidden_layer1[i].get_left(), color = COLOR_WEIGHT, stroke_width = STROKE_WEIGHT) for j in input_layer for i in range(PERCEPTRONS_HIDDEN_LAYER1))
        weights_hidden2.add(Line(j.get_right(), hidden_layer2[i].get_left(), color = COLOR_WEIGHT, stroke_width = STROKE_WEIGHT) for j in hidden_layer1 for i in range(PERCEPTRONS_HIDDEN_LAYER2))
        weights_output.add(Line(j.get_right(), output_layer[i].get_left(), color = COLOR_WEIGHT, stroke_width = STROKE_WEIGHT) for j in hidden_layer2 for i in range(NUM_OUTPUTS))

        # Samples constants
        matrix_squares_colored = DataMatrix(data = pixels_normalized[1], num_rows = NUM_ROWS_MNIST, cell_width = SIZE_MNIST_EXAMPLE_SIDE, cell_height = SIZE_MNIST_EXAMPLE_SIDE, stroke_width = 0.5, color_mode = True).next_to(input_layer, LEFT, buff = MARGIN_EXAMPLE_INPUT)

        self.play(FadeIn(input_layer))
        self.play(AnimationGroup(sucession_animations(FadeIn, [hidden_layer1, hidden_layer2, output_layer], 0.2)))
        self.play(Create(weights_hidden1))
        self.play(Create(weights_hidden2))
        self.play(Create(weights_output))
        self.wait(1)

        box_input_layer = Rectangle(height = input_layer.get_top()[1] - input_layer.get_bottom()[1] + PADDING_SURROUNDING_BOXES, width = input_layer.get_right()[0] - input_layer.get_left()[0] + PADDING_SURROUNDING_BOXES).set_stroke(WHITE, width = 2).move_to(input_layer)
        text_input_layer = MarkupText("Camada de Entrada", font_size = LABELS_FONT_LAYERS, font = FONT_TYPE).next_to(box_input_layer, direction = UP, buff = 0.2)

        box_hidden_layers = Rectangle(height = hidden_layer1.get_top()[1] - hidden_layer1.get_bottom()[1] + PADDING_SURROUNDING_BOXES, width = hidden_layer2.get_right()[0] - hidden_layer1.get_left()[0] + PADDING_SURROUNDING_BOXES).set_stroke(WHITE, width = 2).move_to((hidden_layer1.get_corner(UL) + hidden_layer2.get_corner(DR)) / 2)
        text_hidden_layer = MarkupText("Camadas Ocultas", font_size = LABELS_FONT_LAYERS, font = FONT_TYPE).next_to(box_hidden_layers, direction = UP, buff = 0.2)

        box_output_layer = Rectangle(height = output_layer.get_top()[1] - output_layer.get_bottom()[1] + PADDING_SURROUNDING_BOXES, width = output_layer.get_right()[0] - output_layer.get_left()[0] + PADDING_SURROUNDING_BOXES).set_stroke(WHITE, width = 2).move_to(output_layer)
        text_output_layer = MarkupText("Camada de Saída", font_size = LABELS_FONT_LAYERS, font = FONT_TYPE).next_to(box_output_layer, direction = UP, buff = 0.2)

        #self.play(DrawBorderThenFill(box_input_layer), FadeIn(text_input_layer))
        #self.play(DrawBorderThenFill(box_hidden_layers), FadeIn(text_hidden_layer))
        #self.play(DrawBorderThenFill(box_output_layer), FadeIn(text_output_layer))

        #self.play(FadeOut(box_input_layer), FadeOut(text_input_layer), FadeOut(box_hidden_layers), FadeOut(text_hidden_layer), FadeOut(box_output_layer), FadeOut(text_output_layer))


        self.play(FadeIn(matrix_squares_colored))
        self.play(Transform(matrix_squares_colored, input_layer))

        self.play(add_intervals([animate_half_lines(weights_hidden1, line_width = STROKE_WEIGHT),
                       animate_neurons(hidden_layer1),
                       animate_half_lines(weights_hidden2, line_width = STROKE_WEIGHT),
                       animate_neurons(hidden_layer2),
                       animate_half_lines(weights_hidden2, line_width = STROKE_WEIGHT),
                       animate_output(output_layer, labels[1])]))
        self.play()
        self.play()
        self.wait(2)

Manim Community v0.19.0

[08/12/25 22:30:36] INFO     Animation 0 : Using cached data (hash :                           ]8;id=754244;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=144972;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3529212410_4283996347_223132457)                                                      

[08/12/25 22:30:37] INFO     Animation 1 : Using cached data (hash :                           ]8;id=648849;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=965836;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3040924799_1901359924_1065638087)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=300642;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=565340;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3040924799_1914190057_3724649304)                                                     

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=356295;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=224209;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3040924799_3774835542_8400376)                                                        

[08/12/25 22:30:38] INFO     Animation 4 : Using cached data (hash :                           ]8;id=445513;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=762051;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3040924799_4171231298_1463284839)                                                     

                    INFO     Animation 5 : Using cached data (hash :                           ]8;id=212889;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=812735;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3040924799_4217992463_2294805982)                                                     

[08/12/25 22:30:41] INFO     Animation 6 : Using cached data (hash :                           ]8;id=209525;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=773018;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3040924799_453488500_896657897)                                                       

[08/12/25 22:30:42] INFO     Animation 7 : Using cached data (hash :                           ]8;id=591560;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=502678;file:///usr/local/lib/python3.11/dist-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3040924799_2616920661_1298316742)                                                     

TypeError: Object None cannot be converted to an animation